In [1]:
import tensorflow as tf
import numpy as np
import pickle

In [2]:
data = np.load('music_timesteps.pkl')

In [3]:
data = [d[:, :13] for d in data] # 12 melody notes, plus 'tie'

In [4]:
data = [np.argmax(d, axis=1) for d in data]

In [5]:
def data_gen():
    for d in data:
        yield d

ds = tf.data.Dataset.from_generator(data_gen, tf.int32, tf.TensorShape([None]))
ds = ds.shuffle(buffer_size=128)
ds = ds.batch(1)
x = ds.make_one_shot_iterator().get_next()

In [6]:
sess = tf.InteractiveSession()

In [7]:
lstm_size = 32
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
outputs, states = tf.nn.dynamic_rnn(lstm, tf.one_hot(x, depth=12), dtype=tf.float32)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [10]:
y = tf.layers.dense(outputs, 13 + 1) # melody notes, 'tie' and end of sequence marker
probs = tf.nn.softmax(y)
t = tf.concat([x[1:], [[13]]], axis=0) # the targets are the same as the input,
# but shifted by one timestep and with the end of sequence marker concatenated.
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=t)

In [ ]:
# TODO: add an optimizer
# TODO: look into tf.Estimator